# Rigid Docking

Before you start:

1. Prepare receptor and ligand using AutoDockTools. (or see https://autodock-vina.readthedocs.io/en/latest/docking_basic.html)
2. Prepare or define the search space.

In [1]:
from pathlib import Path

In [2]:
PROTEIN_CLS = "CYTC"
PROTEIN_SID = "FM0"
LIGAND_NAME = "HEC"

WD = Path(f"wd/{PROTEIN_CLS}/{PROTEIN_SID}-{LIGAND_NAME}")
WD.mkdir(parents=True, exist_ok=True)

# You need to prepare the following:
PROTEIN_PDBQT = WD / f"{PROTEIN_SID}.pdbqt"
LIGAND_PDBQT = WD / f"{LIGAND_NAME}.pdbqt"
assert PROTEIN_PDBQT.exists() and LIGAND_PDBQT.exists()

# And define the following search space:
SPACE_CONF = WD / "vina.conf"

# Docking settings:
NUM_MODES = 25
ENERGY_RANGE = 5
EXHAUSTIVENESS = 20

OUTPUT_PDBQT = WD / f"{PROTEIN_SID}-{LIGAND_NAME}.docked.pdbqt"

In [3]:
!cd {WD} && vina --receptor {PROTEIN_PDBQT.name} --ligand {LIGAND_PDBQT.name} --config {SPACE_CONF.name} --out {OUTPUT_PDBQT.name} --num_modes {NUM_MODES} --energy_range {ENERGY_RANGE} --exhaustiveness {EXHAUSTIVENESS} | tee vina.log

#################################################################
# If you used AutoDock Vina in your work, please cite:          #
#                                                               #
# O. Trott, A. J. Olson,                                        #
# AutoDock Vina: improving the speed and accuracy of docking    #
# with a new scoring function, efficient optimization and       #
# multithreading, Journal of Computational Chemistry 31 (2010)  #
# 455-461                                                       #
#                                                               #
# DOI 10.1002/jcc.21334                                         #
#                                                               #
# Please see http://vina.scripps.edu for more information.      #
#################################################################

Detected 24 CPUs
Reading input ... done.
Setting up the scoring function ... done.
Analyzing the binding site ... done.
Using random seed: -1

In [4]:
OUTPUT_LIGAND_PREFIX = f"{LIGAND_NAME}.docked."

!cd {WD} && vina_split --input {OUTPUT_PDBQT.name} --ligand {OUTPUT_LIGAND_PREFIX}

Prefix for flexible side chains will be FM0-HEC.docked_flex_


In [5]:
from pymol import cmd

for ligand_pdbqt in WD.glob(f"{OUTPUT_LIGAND_PREFIX}*.pdbqt"):
    cmd.delete('all')
    cmd.reinitialize()
    cmd.load(PROTEIN_PDBQT, "RECEPTOR")
    cmd.load(ligand_pdbqt, "LIGAND")
    cmd.create("COMPLEX", "RECEPTOR or LIGAND")
    cmd.save(f"{WD}/{PROTEIN_SID}-{ligand_pdbqt.stem}.pdb", "COMPLEX")